In [1]:
from pyspark.sql import SparkSession

# Kubernetes dashboard > Config And Storage > Secrets > s3a-env-configuration
#http://192.168.29.16:9090/browser/test login() :- E2XLJVLJHRZVUVLR & ZQ5BDKSIGDTIUI0CSRZYW22DRWKCTZLE

#Kubernetes dashboard > Service > Services > 
#Name (s3a-hl) 
#Labels(v1.min.io/tenant: s3a) Type(ClusterIP) Cluster IP(none) and Internal Endpoints(s3a-hl.data-platform:9000 TCP & s3a-hl.data-platform:0 TCP)
#http://s3a-hl.data-platform:9000

#http://192.168.29.16:9090 go to Minio console then create Object Browser and upload csv file and then create Access Keys
# "accessKey":"qUqE7TK7Xhkw9hS8DS4b","secretKey":"MeJYIY6s4X8P0SjYF7AsFkgf9AePLu837gndpAKS"

#To access spark cluser and jupyterlab.
#Jupyter Notebook    :- http://localhost:8888/lab/ :- no need token. 
#Jupyter Nodtebook   :- http://localhost:8888/lab/tree/K8s_code_sparkmaster.ipynb.
#Spark Master UI     :- http://10.43.175.33:8080/  spark-master-svc.data-platform:7077
#Application ID Logs :- http://10.42.3.220:8081/logPage/?appId=app-20240830140856-0003&executorId=0&logType=stderr

#For Driver host and driver bindaddress : Go to kubernetes dashboard(Namespace:-Data-platform) > 
#Workloads>Pods>jupiter-spark-f445c778b-bbkcd>Resource information>Node(raspberry)>Status(Running)>Ip(10.42.0.68)

minio_endpoint = "http://s3a-hl.data-platform:9000"  # Ensure this matches your setup s3a-hl.data-platform:9000
minio_access_key = "ykh5O35wc7imnHGw8oqK"
minio_secret_key = "xHsyoAqFE5exvOnfCT9uMcMzvAh73kZPAqrvy8mF"
minio_bucket = "test"

spark = SparkSession.builder \
    .appName("MinIOs") \
    .config('spark.master','spark://spark-master-svc.data-platform:7077')\
    .config("spark.driver.host", "10.42.0.66")\
    .config("spark.driver.bindAddress", "10.42.0.66")\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

In [2]:
dataframe = spark.read.json('s3a://test/orders.json')

average = dataframe.agg({'amount':'avg'})

average.write.format("csv").option("header", "true").save("s3a://test/json/")

average.show()


+-----------+
|avg(amount)|
+-----------+
|     2500.0|
+-----------+



In [3]:
spark.stop()